#### Simulates a simplified airport security system at a busy airport. 
#### Passengers arrive according to a Poisson distribution with λ1 = 5 per minute (i.e., mean interarrival rate µ1 = 0.2 minutes) to the ID/boarding-pass check queue, where there are several servers who each have exponential service time with mean rate µ2 = 0.75 minutes. [Hint: model them as one block that has more than one resource.]  where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1 minute).


In [16]:
# import packages, set queing params
import random
import simpy
import numpy as np
from math import floor

arrival_lambda_1 = 5
arrival_mu = .2
server_mu = .75
scanner_lower = 0.5
scanner_upper = 1
max_wait = 15
random.seed(123)  # This helps reproducing the results

# dictionary for storing avg wait times
avg_wait_times = {}


In [22]:
for i in range(2, 20):
    num_scanners = i + 1
    # for simplicity lets assume they have 2x the scanners as id controllers, as most imports have
    num_controllers = floor(num_scanners/2)
    sim_time = 360
    wait_time = []
    
    # airport class inspired by https://simpy.readthedocs.io/en/latest/examples/carwash.html
    class Airport(object):
        def __init__(self, env, controls, scanners):
            self.env = env
            self.controller = simpy.Resource(env, controls)
            self.scanner = simpy.Resource(env, scanners)


        def id_control(self, passenger):
            yield self.env.timeout(random.expovariate(server_mu))
        
        
        
        def scan(self, passenger):
            yield self.env.timeout(random.uniform(scanner_lower, scanner_upper))

        
    def passenger(env, name, airport):
        with airport.controller.request() as request:
            start = env.now
            yield request
            yield env.process(airport.id_control(name))

       
        with airport.scanner.request() as request:
            yield request
            yield env.process(airport.scan(name))
            wait_time.append(env.now - start)



    def setup(env, controls, scanners):
        # create airport
        airport = Airport(env, controls, scanners)
    
        for i in range(1):
            env.process(passenger(env, 'Passenger %d' % i, airport))

    
        while True:
            arrival_int = random.expovariate(arrival_lambda_1)
            yield env.timeout(arrival_int)
            i += 1
            env.process(passenger(env, 'Passenger %d' % i, airport))
        

    # Setup and start the simulation
    #print('People are arriving...omg!')
    
    # Create an environment and start the setup process
    env = simpy.Environment()
    #env.process(setup(env, num_controllers, num_scanners))
    env.process(setup(env, num_controllers, num_scanners))


    # Execute!
    env.run(until=sim_time)
    
    trial_wait_time = np.mean(wait_time)
    print("they waited an average of {}. what sucksers! with {} id contorls and {} scanners".format(
        trial_wait_time, num_controllers, num_scanners))
    
    #lets save this for later
    avg_wait_times[i + 1] = np.mean(trial_wait_time)
    
          
print(avg_wait_times) 

they waited an average of 154.6393456136308. what sucksers! with 1 id contorls and 3 scanners
they waited an average of 134.87393117657425. what sucksers! with 2 id contorls and 4 scanners
they waited an average of 127.40072005527504. what sucksers! with 2 id contorls and 5 scanners
they waited an average of 102.11171783127156. what sucksers! with 3 id contorls and 6 scanners
they waited an average of 93.75521276759598. what sucksers! with 3 id contorls and 7 scanners
they waited an average of 82.48786185557276. what sucksers! with 4 id contorls and 8 scanners
they waited an average of 71.12557317723198. what sucksers! with 4 id contorls and 9 scanners
they waited an average of 38.40543630383313. what sucksers! with 5 id contorls and 10 scanners
they waited an average of 47.59949183720798. what sucksers! with 5 id contorls and 11 scanners
they waited an average of 16.085267842704923. what sucksers! with 6 id contorls and 12 scanners
they waited an average of 25.948399902803725. what su

#### Looks like 7 id controllers and 14 scanners did the trick!